In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 23733464
paper_name = 'islahudin_avery_2013' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [26]:
original_data = pd.read_excel('raw_data/zac999102061sd2.xlsx', sheet_name='Initial CQ Screen', skiprows=3)

In [27]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 5664 x 13


In [28]:
original_data.head()

,ORF,Plate,Row,Col,Control A,Control B,Chloroquine A,Chloroquine B,MEAN Control,MEAN +Chloroquine,Growth Ratio (GR),Plate Median,Adjusted GR
0,EMPTY,1,A,1,0.117,0.116,0.110,0.105,0.1165,0.1075,SLOW,1.277311,SLOW
1,YHL047C,1,A,2,0.471,0.443,0.432,0.389,0.4570,0.4105,1.11328,1.277311,0.871578
2,YHL046C,1,A,3,0.736,0.509,0.392,0.424,0.6225,0.4080,1.52574,1.277311,1.19449
3,YHL045W,1,A,4,0.667,0.461,0.520,0.380,0.5640,0.4500,1.25333,1.277311,0.981228
4,YHL044W,1,A,5,0.117,0.119,0.107,0.107,0.1180,0.1070,SLOW,1.277311,SLOW


In [29]:
original_data['ORF'] = original_data['ORF'].astype(str)

In [30]:
# Eliminate all white spaces & capitalize
original_data['ORF'] = clean_orf(original_data['ORF'])

In [31]:
# Translate to ORFs 
original_data['ORF'] = translate_sc(original_data['ORF'], to='orf')

In [32]:
to_drop = original_data.loc[original_data['ORF']=='EMPTY',]

In [33]:
original_data.drop(index=to_drop.index, inplace=True)

In [34]:
# Make sure everything translated ok
t = looks_like_orf(original_data['ORF'])
print(original_data.loc[~t,])

                  ORF  Plate Row Col  Control A  Control B  Chloroquine A  \
index_input                                                                 
5604         WTBY4743     71   D   1      0.082      0.081          0.085   
5605         WTBY4743     71   D   2      0.083      0.081          0.083   

             Chloroquine  B  MEAN Control  MEAN +Chloroquine  \
index_input                                                    
5604                  0.084        0.0815             0.0845   
5605                  0.083        0.0820             0.0830   

            Growth Ratio (GR)  Plate Median Adjusted GR  
index_input                                              
5604                     SLOW           NaN        SLOW  
5605                     SLOW           NaN        SLOW  


In [35]:
original_data = original_data.loc[t,:]

In [36]:
# Eliminate the slow growin strains for which no accurate growth ratio could be calculated
original_data = original_data.loc[original_data['Adjusted GR']!='SLOW',]

In [37]:
original_data.shape

(4394, 13)

In [38]:
# Reverse the growth ratio so that lower values correspond to decreased growth and viceversa (originally, GR is reported as untreated vs treated)
original_data['GR2'] = 1 / original_data['Growth Ratio (GR)']

In [39]:
# Normalize by plate median (as done oridinally)
def normalize_by_plate_median(plate_data):
    plate_median = plate_data['GR2'].median()
    plate_data['GR2_adjusted'] = plate_data['GR2'] / plate_median
    return plate_data

In [40]:
original_data2 = original_data.groupby('Plate').apply(normalize_by_plate_median)

In [41]:
original_data2.head()

,ORF,Plate,Row,Col,Control A,Control B,Chloroquine A,Chloroquine B,MEAN Control,MEAN +Chloroquine,Growth Ratio (GR),Plate Median,Adjusted GR,GR2,GR2_adjusted
index_input,,,,,,,,,,,,,,,
1,YHL047C,1,A,2,0.471,0.443,0.432,0.389,0.4570,0.4105,1.11328,1.277311,0.871578,0.898249,1.14734
2,YHL046C,1,A,3,0.736,0.509,0.392,0.424,0.6225,0.4080,1.52574,1.277311,1.19449,0.655422,0.837177
3,YHL045W,1,A,4,0.667,0.461,0.520,0.380,0.5640,0.4500,1.25333,1.277311,0.981228,0.797872,1.01913
5,YHL043W,1,A,6,0.659,0.576,0.402,0.306,0.6175,0.3540,1.74435,1.277311,1.36564,0.573279,0.732256
7,YHL041W,1,A,8,0.617,0.428,0.304,0.332,0.5225,0.3180,1.64308,1.277311,1.28636,0.608612,0.777387


In [42]:
original_data2.set_index('ORF', inplace=True)
original_data2.index.name = 'orf'

In [43]:
original_data2['data'] = original_data2['GR2_adjusted'].astype(float)
original_data2 = original_data2[['data']].copy()

In [44]:
original_data2 = original_data2.groupby(original_data2.index).mean()

In [46]:
original_data2.shape

(4251, 1)

# Prepare the final dataset

In [50]:
data = original_data2.copy()

In [51]:
dataset_ids = [16532]
datasets = datasets.reindex(index=dataset_ids)

In [52]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [53]:
data.head()

dataset_id,16532
data_type,value
orf,
YAL002W,1.004662
YAL004W,0.783971
YAL005C,1.148024
YAL007C,0.755870
YAL008W,0.836115


## Subset to the genes currently in SGD

In [55]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 23


In [56]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16532
,data_type,value
gene_id,orf,
2,YAL002W,1.004662
1863,YAL004W,0.783971
4,YAL005C,1.148024
5,YAL007C,0.755870
6,YAL008W,0.836115


# Normalize

In [57]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [58]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [59]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id          16532          
data_type           value    valuez
gene_id orf                        
2       YAL002W  1.004662  0.070363
1863    YAL004W  0.783971 -1.110923
4       YAL005C  1.148024  0.837730
5       YAL007C  0.755870 -1.261342
6       YAL008W  0.836115 -0.831813

# Print out

In [60]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [61]:
from IO.save_data_to_db3 import *

In [62]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 23733464...
Inserting the new data...


100%|██████████| 1/1 [00:06<00:00,  6.43s/it]

Updating the data_modified_on field...
